# init

(1)接受参数  
(2)创建学习层、线性转换层等  
(3)重设参数 def reset_parameters(self):

# forward()

1.规范检查  
2.消息传递  
3.更新(norm ReLU)  

# 异构图 构建HeteroGraphConv

HeteroGraphConv 的工作原理如下：

它接受一个异质图和一个节点特征字典作为输入  
节点特征字典的键是节点类型，值是对应节点类型的特征矩阵  

对于图中的每种边类型，HeteroGraphConv 应用一个用户定义的卷积模块（例如，GraphConv、SAGEConv 等）来更新源节点和目标节点的特征。  
它聚合来自不同边类型的消息，为每种节点类型生成新的特征矩阵  
最后，它返回一个包含所有节点类型的新特征矩阵的字典  

In [13]:
import dgl
import torch
import dgl.nn as dglnn
from dgl.nn import HeteroGraphConv

In [14]:
# 定义一个异质图卷积层
conv = HeteroGraphConv({
    'follows': dglnn.GraphConv(10, 20),
    'plays': dglnn.GraphConv(10, 20)
})

# 创建一个DGL异质图对象g和一个节点特征字典features
# 假设我们有以下边关系
edges_follows = torch.tensor([0, 1, 2, 3])  # 源节点
edges_plays = torch.tensor([0, 1, 2, 3])    # 目标节点
edges_types = torch.tensor([0, 0, 1, 1])    # 边类型（0表示follows，1表示plays）

# 创建异质图
g = dgl.heterograph({
    ('user', 'follows', 'user'): (edges_follows, edges_follows),
    ('user', 'plays', 'game'): (edges_plays, edges_plays)
})


# 创建节点特征
features = {
    'user': torch.randn((g.num_nodes('user'), 10)),
    'game': torch.randn((g.num_nodes('game'), 10))
}

# 使用异质图卷积层进行前向传播
new_features = conv(g, features)
print(new_features)

{'game': tensor([[-0.4186, -0.0787,  0.2078,  0.1346,  0.0228, -0.3238,  0.4192,  0.1570,
         -1.0262,  1.1738, -0.2227,  0.4517, -0.6732, -0.5751,  0.4685, -1.1541,
          0.9681,  0.3694,  0.0205, -0.5689],
        [ 0.9645,  0.2925,  1.6513,  0.6263, -0.2770,  0.3604,  0.3153,  0.0377,
         -0.7422, -1.0357, -1.5241, -0.3054, -0.5326,  0.7953,  0.2911, -0.6869,
          0.0489,  1.5596,  0.1493, -1.2439],
        [ 0.3110, -2.4573, -0.3757,  1.0528,  0.9912,  0.7038,  1.6694, -1.0276,
         -1.6601,  0.8026,  0.3320,  0.2934, -0.6434, -1.3492,  0.3117, -0.6945,
          0.6976,  1.4285, -1.3700, -1.3188],
        [ 0.3013, -1.4782, -0.5431,  0.9906,  0.6119, -0.8032,  1.7863, -0.7437,
          0.5779,  0.3741,  1.7158, -1.5371, -0.0712, -0.7305,  0.3376,  0.0762,
          0.2163,  0.4033, -0.7931,  0.4584]], grad_fn=<SumBackward1>), 'user': tensor([[ 0.0293,  0.1164, -0.4904,  0.5563, -0.2621,  0.0718,  0.2468,  0.8792,
         -0.3826, -0.4367,  0.0502,  0.0647,